In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib
import os

In [ ]:
import pandas as pd

# Load the dataset (example filename)
df = pd.read_csv("/content/drive/MyDrive/INTERN/companies_sorted.csv") #, usecols=["name", "domain", "industry", "description"])



In [ ]:
  df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7173426 entries, 0 to 7173425
Data columns (total 11 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Unnamed: 0                 int64  
 1   name                       object 
 2   domain                     object 
 3   year founded               float64
 4   industry                   object 
 5   size range                 object 
 6   locality                   object 
 7   country                    object 
 8   linkedin url               object 
 9   current employee estimate  int64  
 10  total employee estimate    int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 602.0+ MB


In [ ]:
df.shape

(7173426, 11)

In [ ]:
df.head(2)

,Unnamed: 0,name,domain,year founded,industry,size range,locality,country,linkedin url,current employee estimate,total employee estimate
0,5872184,ibm,ibm.com,1911.0,information technology and services,10001+,"new york, new york, united states",united states,linkedin.com/company/ibm,274047,716906
1,4425416,tata consultancy services,tcs.com,1968.0,information technology and services,10001+,"bombay, maharashtra, india",india,linkedin.com/company/tata-consultancy-services,190771,341369


In [ ]:
df.describe()

,Unnamed: 0,year founded,current employee estimate,total employee estimate
count,7.173426e+06,3.566446e+06,7.173426e+06,7.173426e+06
mean,3.586712e+06,2.001747e+03,1.387121e+01,3.225438e+01
std,2.070790e+06,2.096627e+01,3.545919e+02,8.741556e+02
min,0.000000e+00,1.451000e+03,0.000000e+00,1.000000e+00
25%,1.793356e+06,1.999000e+03,1.000000e+00,1.000000e+00
50%,3.586712e+06,2.009000e+03,1.000000e+00,2.000000e+00
75%,5.380069e+06,2.013000e+03,4.000000e+00,7.000000e+00
max,7.173425e+06,2.103000e+03,2.740470e+05,7.169060e+05


In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
name,3
domain,1650621
year founded,3606980
industry,290003
size range,0
locality,2508825
country,2349207
linkedin url,0
current employee estimate,0


In [ ]:
# Load your huge dataset
df = pd.read_csv("/content/drive/MyDrive/INTERN/companies_sorted.csv")

# Drop rows with no industry
df = df.dropna(subset=["industry"])

# Optional: keep only top N industries
top_industries = df["industry"].value_counts().nlargest(10).index
df = df[df["industry"].isin(top_industries)]

# Create synthetic description
df["Description"] = (
    "A " + df["industry"].fillna("company") +
    " company based in " + df["country"].fillna("unknown") +
    " operating in the " + df["size range"].fillna("N/A") +
    " segment."
)

# Create final DataFrame
df_final = df[["Description", "industry"]].rename(columns={"industry": "Industry"})

# Drop nulls just in case
df_final = df_final.dropna()

# Create the 'data' directory if it doesn't exist
import os
os.makedirs("data", exist_ok=True)

# Save 100,000 rows for training
df_final.sample(100_000, random_state=42).to_csv("data/sample_leads.csv", index=False)

print("✅ sample_leads.csv ready for training")

✅ sample_leads.csv ready for training


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1437169 entries, 0 to 7173421
Data columns (total 2 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   Description  1437169 non-null  object
 1   Industry     1437169 non-null  object
dtypes: object(2)
memory usage: 32.9+ MB


In [ ]:
# Step 1: Load and clean data
df = df_final
df.dropna(subset=["Description", "Industry"], inplace=True)
df = df[df["Description"].str.len() > 10]  # Drop short descriptions

In [ ]:
# Optional: limit to top 5 industries for balance
top_industries = df["Industry"].value_counts().nlargest(5).index
df = df[df["Industry"].isin(top_industries)]

In [ ]:
# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(df["Description"], df["Industry"], test_size=0.2, random_state=42)

# Step 3: Build pipeline
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english", max_features=10000)),
    ("select", SelectKBest(score_func=chi2, k=1000)),
    ("clf", LogisticRegression(max_iter=300))
])

# Step 4: GridSearchCV
params = {
    "clf": [LogisticRegression(max_iter=300), MultinomialNB()],
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [3, 5],
    "tfidf__max_df": [0.8, 0.9]
}

In [ ]:
grid = GridSearchCV(pipeline, param_grid=params, cv=3, verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)


Fitting 3 folds for each of 16 candidates, totalling 48 fits


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=1000 is greater than n_features=259. All the features will be returned.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=10000,
                                                        stop_words='english')),
                                       ('select',
                                        SelectKBest(k=1000,
                                                    score_func=<function chi2 at 0x7e3435c645e0>)),
                                       ('clf',
                                        LogisticRegression(max_iter=300))]),
             n_jobs=-1,
             param_grid={'clf': [LogisticRegression(max_iter=300),
                                 MultinomialNB()],
                         'tfidf__max_df': [0.8, 0.9], 'tfidf__min_df': [3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=2)

In [ ]:
# Step 5: Evaluate
y_pred = grid.predict(X_test)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))



📊 Classification Report:
                                     precision    recall  f1-score   support

                       construction       1.00      1.00      1.00     51352
information technology and services       1.00      1.00      1.00     90731
              management consulting       1.00      1.00      1.00     38707
          marketing and advertising       1.00      1.00      1.00     68060
                        real estate       1.00      1.00      1.00     38584

                           accuracy                           1.00    287434
                          macro avg       1.00      1.00      1.00    287434
                       weighted avg       1.00      1.00      1.00    287434



In [ ]:
# Step 6: Save model
best_pipeline = grid.best_estimator_
vectorizer = best_pipeline.named_steps["tfidf"]
classifier = best_pipeline.named_steps["clf"]

os.makedirs("model", exist_ok=True)
joblib.dump(classifier, "model/lead_classifier.pkl")
joblib.dump(vectorizer, "model/tfidf_vectorizer.pkl")
print("\n✅ Model and vectorizer saved to /model/")



✅ Model and vectorizer saved to /model/


In [ ]:
# Assuming df_final is a pandas DataFrame
df_final.to_csv('df_final.csv')

In [2]:
# Step 1: Load and clean data
df = pd.read_csv("/content/df_final.csv")
df.dropna(subset=["Description", "Industry"], inplace=True)
df = df[df["Description"].str.len() > 10]  # Drop short descriptions

In [3]:
# Optional: limit to top 5 industries for balance
top_industries = df["Industry"].value_counts().nlargest(5).index
df = df[df["Industry"].isin(top_industries)]

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split, GridSearchCV

# Step 2: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df["Description"], df["Industry"], test_size=0.2, random_state=42
)

# Step 3: Build pipeline using only MultinomialNB
pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(stop_words="english", max_features=10000)),
    ("select", SelectKBest(score_func=chi2, k=1000)),
    ("clf", MultinomialNB())
])

# Step 4: GridSearchCV (optimize only TF-IDF params)
params = {
    "tfidf__ngram_range": [(1, 1), (1, 2)],
    "tfidf__min_df": [3, 5],
    "tfidf__max_df": [0.8, 0.9]
}



In [5]:
grid = GridSearchCV(pipeline, param_grid=params, cv=3, verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)


Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.11/dist-packages/sklearn/feature_selection/_univariate_selection.py:783: UserWarning: k=1000 is greater than n_features=254. All the features will be returned.
  warnings.warn(


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_features=10000,
                                                        stop_words='english')),
                                       ('select',
                                        SelectKBest(k=1000,
                                                    score_func=<function chi2 at 0x7b1f164fde40>)),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'tfidf__max_df': [0.8, 0.9], 'tfidf__min_df': [3, 5],
                         'tfidf__ngram_range': [(1, 1), (1, 2)]},
             verbose=2)

In [6]:
# Step 5: Evaluate
y_pred = grid.predict(X_test)
print("\n📊 Classification Report:")
print(classification_report(y_test, y_pred))



📊 Classification Report:
                                     precision    recall  f1-score   support

                  computer software       1.00      1.00      1.00     23042
                       construction       1.00      1.00      1.00     30747
information technology and services       1.00      1.00      1.00     56934
          marketing and advertising       1.00      1.00      1.00     41100
                        real estate       1.00      1.00      1.00     24348

                           accuracy                           1.00    176171
                          macro avg       1.00      1.00      1.00    176171
                       weighted avg       1.00      1.00      1.00    176171

